# Generic Conditional Laws for Random-Fields - via:

## Universal $\mathcal{P}_1(\mathbb{R})$-Deep Neural Model $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$.

---

By: [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

#### Mode:
Software/Hardware Testing or Real-Deal?

In [1]:
trial_run = False

### Simulation Method:

In [2]:
# Random DNN
method = "Random_DNN"

# Random DNN internal noise
# method = "Random_DNN_internal_noise"

---
# Training Algorithm:
---
- Random $\delta$-bounded partition on input space,
- Train deep classifier on infered classes.
---
---
---
## Notes - Why the procedure is so computationally efficient?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

## Load Auxiliaries

In [3]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Import time separately
import time


# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"


### Set Seed
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready


## Meta-Parameters

### Simulation

## Problem Dimension

In [4]:
problem_dim = 100
width = 300

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [5]:
train_test_ratio = .2
N_train_size = 10**2

Monte-Carlo Paramters

In [6]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**3

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [7]:
# Hyper-parameters of Cover
delta = 0.01
Proportion_per_cluster = .05

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

# Simulate from: $Y=f(X,U)$ 
- Random DNN (internal noise): 
    - $f(X,U) = f_{\text{unknown}}(X+U)$
- Random DNN: 
    - $f(X,U) = f_{\text{unknown}}(X)+U$
    
*Non-linear dependance on exhaugenous noise.*

In [8]:
# Hard
W_2a = np.random.uniform(size=np.array([1,width]),low=-.5,high=.5)
W_1a = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
def f_unknown(x):
    x_internal = x.reshape(-1,)
    x_internal = np.matmul(W_1a,x_internal)
    x_internal = np.matmul(W_2a,np.cos(x_internal))
    return x_internal

In [30]:
def Simulator(x_in):
    var = np.sqrt(np.sum(x_in**2))
    # Pushforward
    f_x = f_unknown(x_in)
    # Apply Noise After
    noise = np.random.laplace(0,var,N_Monte_Carlo_Samples)
    f_x_noise = np.cos(f_x) + noise
    return f_x_noise

## Initialize Data

In [31]:
N_test_size = int(np.round(N_train_size*train_test_ratio,0))

### Initialize Training Data (Inputs)

Try initial sampling-type implementation!  It worked nicely..i.e.: centers were given!

In [32]:
# Get Training Set
X_train = np.random.uniform(size=np.array([N_train_size,problem_dim]),low=.5,high=1.5)

# Get Testing Set
test_set_indices = np.random.choice(range(X_train.shape[0]),N_test_size)
X_test = X_train[test_set_indices,]
X_test = X_test + np.random.uniform(low=-(delta/np.sqrt(problem_dim)), 
                                    high = -(delta/np.sqrt(problem_dim)),
                                    size = X_test.shape)

In [33]:
from sklearn.cluster import KMeans

In [34]:
# Initialize k_means
N_Quantizers_to_parameterize = int(round(Proportion_per_cluster*X_train.shape[0]))
kmeans = KMeans(n_clusters=N_Quantizers_to_parameterize, random_state=0).fit(X_train)
# Get Classes
Train_classes = np.array(pd.get_dummies(kmeans.labels_))
# Get Center Measures
Barycenters_Array_x = kmeans.cluster_centers_

### Get Barycenters
*Here we make the assumption that we can directly resample $f(X=x,U)$ if necessary...or that it is available as part of the dataset.*

In [35]:
for i in tqdm(range(Barycenters_Array_x.shape[0])):
    # Put Datum
    Bar_x_loop = Barycenters_Array_x[i,]
    # Product Monte-Carlo Sample for Input
    Bar_y_loop = (Simulator(Bar_x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Barycenters_Array = Bar_y_loop
    else:
        Barycenters_Array = np.append(Barycenters_Array,Bar_y_loop,axis=0)

100%|██████████| 5/5 [00:00<00:00, 389.46it/s]


### Initialize Training Data (Outputs)

#### Get Training Set

In [36]:
for i in tqdm(range(X_train.shape[0])):
    # Put Datum
    x_loop = X_train[i,]
    # Product Monte-Carlo Sample for Input
    y_loop = (Simulator(x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Y_train = y_loop
        Y_train_mean_emp = np.mean(y_loop)
    else:
        Y_train = np.append(Y_train,y_loop,axis=0)
        Y_train_mean_emp = np.append(Y_train_mean_emp,np.mean(y_loop))

100%|██████████| 100/100 [00:00<00:00, 674.59it/s]


#### Get Test Set

In [37]:
# Start Timer
Test_Set_PredictionTime_MC = time.time()

# Generate Data
for i in tqdm(range(X_test.shape[0])):
    # Put Datum
    x_loop = X_test[i,]
    # Product Monte-Carlo Sample for Input
    y_loop = (Simulator(x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Y_test = y_loop
    else:
        Y_test = np.append(Y_test,y_loop,axis=0)
        
# End Timer
Test_Set_PredictionTime_MC = time.time() - Test_Set_PredictionTime_MC

100%|██████████| 20/20 [00:00<00:00, 834.21it/s]


# Train Model

#### Start Timer

In [38]:
# Start Timer
Type_A_timer_Begin = time.time()

### Train Deep Classifier

In this step, we train a deep (feed-forward) classifier:
$$
\hat{f}\triangleq \operatorname{Softmax}_N\circ W_J\circ \sigma \bullet \dots \sigma \bullet W_1,
$$
to identify which barycenter we are closest to.

#### Train Deep Classifier

Re-Load Packages and CV Grid

In [39]:
# Re-Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Re-Load Classifier Function(s)
exec(open('Helper_Functions.py').read())

Deep Feature Builder - Ready
Deep Classifier - Ready


Train Deep Classifier

In [40]:
print("==========================================")
print("Training Classifer Portion of Type-A Model")
print("==========================================")

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [problem_dim]
param_grid_Deep_Classifier['output_dim'] = [N_Quantizers_to_parameterize]

# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier, timer_output = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter = n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train, 
                                                                                                        y_train = Train_classes,
                                                                                                        X_test = X_test)

print("===============================================")
print("Training Classifer Portion of Type Model: Done!")
print("===============================================")

Training Classifer Portion of Type-A Model
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   11.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   19.5s
[Parallel(n_jobs=4)]: Done  16 out of  20 | elapsed:   29.3s remaining:    7.3s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   40.1s finished


Epoch 1/200
4/4 [==============================] - 0s 2ms/step - loss: 1.6680 - accuracy: 0.2000
Epoch 2/200
4/4 [==============================] - 0s 4ms/step - loss: 1.5204 - accuracy: 0.3100
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 1.4424 - accuracy: 0.3300
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 1.3924 - accuracy: 0.3500
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 1.3104 - accuracy: 0.4900
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2508 - accuracy: 0.6900
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1657 - accuracy: 0.8300
Epoch 8/200
4/4 [==============================] - 0s 4ms/step - loss: 1.0618 - accuracy: 0.7800
Epoch 9/200
4/4 [==============================] - 0s 4ms/step - loss: 1.0049 - accuracy: 0.7100
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9158 - accuracy: 0.8200
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 94/200
4/4 [===============

4/4 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 2ms/step - loss: 9.9855e-04 - accuracy: 1.0000
Epoch 177/200
4/4 [=

#### Get Predicted Quantized Distributions
- Each *row* of "Predicted_Weights" is the $\beta\in \Delta_N$.
- Each *Column* of "Barycenters_Array" denotes the $x_1,\dots,x_N$ making up the points of the corresponding empirical measures.

In [41]:
# Initialize Empirical Weights
empirical_weights = (np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples).reshape(-1,)

for i in range(N_Quantizers_to_parameterize):
    if i == 0:
        points_of_mass = Barycenters_Array[i,]
    else:
        points_of_mass = np.append(points_of_mass,Barycenters_Array[i,])

In [42]:
# Get Noisless Mean
direct_facts = np.apply_along_axis(f_unknown, 1, X_train)
direct_facts_test = np.apply_along_axis(f_unknown, 1, X_test)

#### Get Training Errors

In [43]:
print("#--------------------#")
print(" Get Training Error(s)")
print("#--------------------#")
for i in tqdm(range((X_train.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop = np.repeat(predicted_classes_train[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b = b_loop
        else:
            b = np.append(b,b_loop)
        b = b.reshape(-1,1)
        b = b
    b = np.array(b,dtype=float).reshape(-1,)
    b = b/N_Monte_Carlo_Samples
    
    # Compute Error(s)
    ## W1
    W1_loop = ot.emd2_1d(points_of_mass,
                         np.array(Y_train[i,]).reshape(-1,),
                         b,
                         empirical_weights)
    
    ## M1
    Mu_hat = np.sum(b*(points_of_mass))
    Mu_MC = np.mean(np.array(Y_train[i,]))
    Mu = direct_facts[i,]
    ### Error(s)
    Mean_loop = (Mu_hat-Mu)
    Mean_loop_MC = (Mu_hat-Mu_MC)
    
    ## M2
    Var_hat = np.sum(((points_of_mass-Mu_hat)**2)*b)
    Var_MC = np.mean(np.array(Y_train[i]-Mu_MC)**2)
    Var = np.mean((direct_facts[i,]-Mu)**2)
    
    ### Error(s)
    Var_loop = np.abs(Var_hat-Var)
    Var_loop_MC = np.abs(Var_MC-Var)
    
    # Update
    if i == 0:
        W1_errors = W1_loop
        Mean_errors =  Mean_loop
        Var_errors = Var_loop
        Mean_errors_MC =  Mean_loop_MC
        Var_errors_MC = Var_loop_MC
        
        
    else:
        W1_errors = np.append(W1_errors,W1_loop)
        Mean_errors =  np.append(Mean_errors,Mean_loop)
        Var_errors = np.append(Var_errors,Var_loop)
        Mean_errors_MC =  np.append(Mean_errors_MC,Mean_loop_MC)
        Var_errors_MC = np.append(Var_errors_MC,Var_loop_MC)
        
print("#-------------------------#")
print(" Get Training Error(s): END")
print("#-------------------------#")

  0%|          | 0/100 [00:00<?, ?it/s]

#--------------------#
 Get Training Error(s)
#--------------------#


100%|██████████| 100/100 [00:00<00:00, 544.36it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#


#### Get Test Errors

In [44]:
print("#----------------#")
print(" Get Test Error(s)")
print("#----------------#")
for i in tqdm(range((X_test.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop_test = np.repeat(predicted_classes_test[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b_test = b_loop_test
        else:
            b_test = np.append(b,b_loop)
        b_test = b_test.reshape(-1,1)
    b_test = np.array(b,dtype=float).reshape(-1,)
    b_test = b/N_Monte_Carlo_Samples
    
    # Compute Error(s)
    ## W1
    W1_loop_test = ot.emd2_1d(points_of_mass,
                         np.array(Y_test[i,]).reshape(-1,),
                         b,
                         empirical_weights)
    
    ## M1
    Mu_hat_test = np.sum(b_test*(points_of_mass))
    Mu_MC_test = np.mean(np.array(Y_test[i,]))
    Mu_test = direct_facts_test[i,]
    ### Error(s)
    Mean_loop_test = (Mu_hat_test-Mu_test)
    Mean_loop_MC_test = (Mu_hat_test-Mu_MC_test)
    
    ## M2
    Var_hat_test = np.sum(((points_of_mass-Mu_hat_test)**2)*b)
    Var_MC_test = np.mean(np.array(Y_test[i]-Mu_MC)**2)
    Var_test = np.mean((direct_facts_test[i,]-Mu)**2)
    
    ### Error(s)
    Var_loop_test = np.abs(Var_hat_test-Var_test)
    Var_loop_MC_test = np.abs(Var_MC_test-Var_test)
    
    # Update
    if i == 0:
        W1_errors_test = W1_loop_test
        Mean_errors_test =  Mean_loop_test
        Var_errors_test = Var_loop_test
        Mean_errors_MC_test =  Mean_loop_MC_test
        Var_errors_MC_test = Var_loop_MC_test
        
        
    else:
        W1_errors_test = np.append(W1_errors_test,W1_loop_test)
        Mean_errors_test =  np.append(Mean_errors_test,Mean_loop_test)
        Var_errors_test = np.append(Var_errors_test,Var_loop_test)
        Mean_errors_MC_test =  np.append(Mean_errors_MC_test,Mean_loop_MC_test)
        Var_errors_MC_test = np.append(Var_errors_MC_test,Var_loop_MC_test)
        
print("#-------------------------#")
print(" Get Training Error(s): END")
print("#-------------------------#")

100%|██████████| 20/20 [00:00<00:00, 587.10it/s]

#----------------#
 Get Test Error(s)
#----------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#


#### Stop Timer

In [45]:
# Stop Timer
Type_A_timer_end = time.time()
# Compute Lapsed Time Needed For Training
Time_Lapse_Model_A = Type_A_timer_end - Type_A_timer_Begin

---
# Benchmarks
---

In [65]:
%run Benchmarks.ipynb
exec(open('CV_Grid.py').read())

Deep Feature Builder - Ready


Implements the Elastic-Net Regression- only predicting mean

In [55]:
# Fit Elastic Net Model
Lin_reg.fit(X_train,Y_train_mean_emp)

# Get Predictions
ENET_predict = Lin_reg.predict(X_train)
ENET_predict_test = Lin_reg.predict(X_test)

# Get Prediction Errors
## Train
for i in range(X_train.shape[0]):
    Mu = direct_facts[i,]
    error_loop = np.abs(Mu-ENET_predict[i])
    if i == 0:
        Mean_errors_ENET = error_loop
    else:
        Mean_errors_ENET = np.append(Mean_errors_ENET,error_loop)
## Test
for i in range(X_test.shape[0]):
    Mu_test = direct_facts[i,]
    error_loop_test = np.abs(Mu-ENET_predict_test[i])
    if i == 0:
        Mean_errors_ENET_test = error_loop_test
    else:
        Mean_errors_ENET_test = np.append(Mean_errors_ENET_test,error_loop_test)

### Kernel Ridge Regression

In [67]:
Xhat_Kridge, Xhat_Kridge_test , relic = get_Kernel_Ridge_Regressor(X_train,X_test,Y_train_mean_emp)


# Get Prediction Errors
## Train
for i in range(X_train.shape[0]):
    Mu = direct_facts[i,]
    error_loop = np.abs(Mu-Xhat_Kridge[i])
    if i == 0:
        Mean_errors_KRidge = error_loop
    else:
        Mean_errors_KRidge = np.append(Mean_errors_KRidge,error_loop)
## Test
for i in range(X_test.shape[0]):
    Mu_test = direct_facts[i,]
    error_loop_test = np.abs(Mu-Xhat_Kridge_test[i])
    if i == 0:
        Mean_errors_KRidge_test = error_loop_test
    else:
        Mean_errors_KRidge_test = np.append(Mean_errors_KRidge_test,error_loop_test)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0127s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0127s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  16 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.1s finished


## Gradient-Boosted Random Forest Regressor

In [57]:
GBRF_y_hat_train, GBRF_y_hat_test, GBRF_model = get_GBRF(X_train,X_test,Y_train_mean_emp)

# Get Prediction Errors
## Train
for i in range(X_train.shape[0]):
    Mu = direct_facts[i,]
    error_loop = np.abs(Mu-GBRF_y_hat_train[i])
    if i == 0:
        Mean_errors_GBRF = error_loop
    else:
        Mean_errors_GBRF = np.append(Mean_errors_GBRF,error_loop)
## Test
for i in range(X_test.shape[0]):
    Mu_test = direct_facts[i,]
    error_loop = np.abs(Mu-GBRF_y_hat_test[i])
    if i == 0:
        Mean_errors_GBRF_test = error_loop_test
    else:
        Mean_errors_GBRF_test = np.append(Mean_errors_GBRF_test,error_loop_test)

# Summary of Mean-Centric Regression Models

In [60]:
Summary_mean_models = pd.DataFrame({"M1":np.array([np.mean(np.abs(Mean_errors)),np.mean(np.abs(Mean_errors_test))]),
                                    "M1_MC":np.array([np.mean(np.abs(Mean_errors_MC)),np.mean(np.abs(Mean_errors_MC_test))]),
                                    "M1_ENET":np.array([np.mean(np.abs(Mean_errors_ENET)),np.mean(np.abs(Mean_errors_ENET_test))]),
                                    "M1_KRidge":np.array([np.mean(np.abs(Mean_errors_KRidge)),np.mean(np.abs(Mean_errors_KRidge_test))]),
                                    "M1_GBRFR":np.array([np.mean(np.abs(Mean_errors_GBRF)),np.mean(np.abs(Mean_errors_GBRF_test))])
                                   },index=["Train","Test"])

Summary_mean_models

,M1,M1_MC,M1_ENET,M1_GBRFR
Train,2.2861E+00,8.6589E-01,2.2543E+00,2.2603E+00
Test,2.0211E+00,8.3213E-01,2.7705E+00,2.7627E+00


## Get Moment Predictions

#### Write Predictions

### Training-Set Result(s): 

In [28]:
#---------------------------------------------------------------------------------------------#
W1_95 = bootstrap(W1_errors, n=1000, func=np.mean)(.95)
W1_99 = bootstrap(W1_errors, n=1000, func=np.mean)(.99)
#---------------------------------------------------------------------------------------------#
Model_Complexity = pd.DataFrame({"N_Centers":N_Quantizers_to_parameterize,
                                 "N_Q":N_Monte_Carlo_Samples,
                                 "N_Params":N_params_deep_classifier,
                                 "Training Time":Time_Lapse_Model_A,
                                 "T_Test/T_Test-MC": (timer_output/Test_Set_PredictionTime_MC),
                                 "Time Test": timer_output,
                                 "Time EM-MC": Test_Set_PredictionTime_MC},index=["Model_Complexity_metrics"])

#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
Summary = pd.DataFrame({"W1":np.array([np.mean(np.abs(W1_errors)),np.mean(np.abs(W1_errors_test))]),
                        "M1":np.array([np.mean(np.abs(Mean_errors)),np.mean(np.abs(Mean_errors_test))]),
                        "M1_MC":np.array([np.mean(np.abs(Mean_errors_MC)),np.mean(np.abs(Mean_errors_MC_test))]),
                        "Var":np.array([np.mean(np.abs(Var_errors)),np.mean(np.abs(Var_errors_test))]),
                        "Var_MC":np.array([np.mean(np.abs(Var_errors_MC)),np.mean(np.abs(Var_errors_MC_test))]),                                             
                        "N_Centers":np.array((N_Quantizers_to_parameterize,N_Quantizers_to_parameterize)),
                        "N_Q":np.array((N_Monte_Carlo_Samples,N_Monte_Carlo_Samples)),
                        "N_Params":np.array((N_params_deep_classifier,N_params_deep_classifier)),
                        "Training Time":np.array((Time_Lapse_Model_A,Time_Lapse_Model_A)),
                        "T_Test/T_Test-MC":np.array(((timer_output/Test_Set_PredictionTime_MC),(timer_output/Test_Set_PredictionTime_MC))),
                        "Problem_Dimension":np.array((problem_dim,problem_dim))
                       },index=["Train","Test"])


# Write Performance Metrics to file #
#-----------------------------------#
pd.set_option('display.float_format', '{:.4E}'.format)
Model_Complexity.to_latex((results_tables_path+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__ModelComplexities.tex"))
pd.set_option('display.float_format', '{:.4E}'.format)
Summary.to_latex((results_tables_path+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"))


#---------------------------------------------------------------------------------------------#
# Update User
print(Model_Complexity)
print(Summary)

                          N_Centers   N_Q  N_Params  Training Time  \
Model_Complexity_metrics          5  1000    152005     4.0156E+01   

                          T_Test/T_Test-MC  Time Test  Time EM-MC  
Model_Complexity_metrics        1.3128E+00 4.8651E-02  3.7058E-02  
              W1         M1      M1_MC        Var     Var_MC  N_Centers   N_Q  \
Train 8.8274E+00 2.0022E+00 2.0097E+00 1.9577E+02 2.1642E+02          5  1000   
Test  6.4414E+00 1.5085E+00 1.6711E+00 2.0322E+02 2.1386E+02          5  1000   

       N_Params  Training Time  T_Test/T_Test-MC  Problem_Dimension  
Train    152005     4.0156E+01        1.3128E+00                100  
Test     152005     4.0156E+01        1.3128E+00                100  


# Update User

## Training Set Prediction Quality

In [29]:
Summary

,W1,M1,M1_MC,Var,Var_MC,N_Centers,N_Q,N_Params,Training Time,T_Test/T_Test-MC,Problem_Dimension
Train,8.8274E+00,2.0022E+00,2.0097E+00,1.9577E+02,2.1642E+02,5,1000,152005,4.0156E+01,1.3128E+00,100
Test,6.4414E+00,1.5085E+00,1.6711E+00,2.0322E+02,2.1386E+02,5,1000,152005,4.0156E+01,1.3128E+00,100


---

---
# Fin
---

---